In [4]:
from keras.layers  import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [5]:
IMAGE_SIZE= [224,224]

In [9]:
train_path = r'C:\Users\Chaitanya Kaul\chest_xray\train'
valid_path = r'C:\Users\Chaitanya Kaul\chest_xray\test' 

In [31]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [32]:
# dont train existing weights
for layer in vgg.layers:
    layer.trainable=False

In [33]:
# useful for getting number of output classes
folders=glob(r'C:\Users\Chaitanya Kaul\chest_xray\train\*') # trying to see gow many classes we have inside our train data

In [34]:
# our layers
x= Flatten()(vgg.output)

In [35]:
prediction= Dense(len(folders),activation='softmax')(x)

#create a model project
model= Model(inputs=vgg.input,outputs=prediction)

In [36]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [37]:
# tell the model what cost and optimization mthod to use
model.compile(
             loss='categorical_crossentropy',
              optimizer='adam',
             metrics=['accuracy'])

In [44]:
# use the image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [46]:
#make sure you provide the same target size as initialized for the image size
training_set = train_datagen.flow_from_directory(r'C:\Users\Chaitanya Kaul\chest_xray\train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 5216 images belonging to 2 classes.


In [48]:

test_set = test_datagen.flow_from_directory(r'C:\Users\Chaitanya Kaul\chest_xray\test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [49]:

# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
163/163 [==============================] - 3783s 23s/step - loss: 0.2203 - accuracy: 0.9206 - val_loss: 1.2384 - val_accuracy: 0.8926
Epoch 2/5
163/163 [==============================] - 3608s 22s/step - loss: 0.1105 - accuracy: 0.9569 - val_loss: 0.9081 - val_accuracy: 0.8782
Epoch 3/5
163/163 [==============================] - 3119s 19s/step - loss: 0.0953 - accuracy: 0.9640 - val_loss: 0.0693 - val_accuracy: 0.8910
Epoch 4/5
163/163 [==============================] - 3207s 20s/step - loss: 0.0953 - accuracy: 0.9626 - val_loss: 0.2771 - val_accuracy: 0.9103
Epoch 5/5
163/163 [==============================] - 2938s 18s/step - loss: 0.0757 - accuracy: 0.9730 - val_loss: 0.3845 - val_accuracy: 0.9199


In [50]:
# saving as a h5 file

import tensorflow as tf

from keras.models import load_model

model.save('model_vgg16.h5')
